In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from astropy import units as u
from astropy.io import ascii
from astroquery.xmatch import XMatch
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord, ICRS, Galactic, Angle
from astropy.time import Time
from tqdm import tqdm
from scipy.stats import norm
import scipy.stats

In [2]:
pi = np.pi
sin = np.sin
cos = np.cos
log10 = np.log10
sqrt = np.sqrt

c=2.99792458*(10**8)
G=6.67428*(10**(-11))
s_mass=G*(1.98892*10**(30))/(c**3)

In [3]:
files = glob.glob('../IPTA_DR1/*/*.par')

In [4]:
def hist_weight(lst):
    return np.ones_like(lst)/len(lst)

def auto_bin(bin_size, min_edge, max_edge):
    #bin_size = 0.05; min_edge = -3.5; max_edge = -1
    N = (max_edge-min_edge)/bin_size; Nplus1 = N + 1
    bin_list = np.linspace(min_edge, max_edge, Nplus1)
    return bin_list

def ang_dist(alpha1, alpha2, delta1, delta2):
    """
    RA (alpha) and DEC (delta) are both in radians.
    returns angular distance in arcseconds.
    """
    alpha1 = alpha1*pi/180
    alpha2 = alpha2*pi/180
    delta1 = delta1*pi/180
    delta2 = delta2*pi/180

    arg = sin(delta1)*sin(delta2)+cos(delta1)*cos(delta2)*cos(alpha1-alpha2)
    theta = np.arccos(arg)*180/pi*3600
    est = np.sqrt(2*(1-arg))
    return theta

# plotting preamble

fig_width_pt = 245.27 #513.17           # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (np.sqrt(5)-1.0)/2.0      # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height =fig_width*golden_mean       # height in inches
fig_size = [fig_width,fig_height]

params = {'backend': 'pdf',
        'axes.labelsize': 7,
        'lines.markersize': 2,
        'font.size': 7,
        'xtick.major.size':6,
        'xtick.minor.size':3,  
        'ytick.major.size':6,
        'ytick.minor.size':3, 
        'xtick.major.width':0.5,
        'ytick.major.width':0.5,
        'xtick.minor.width':0.5,
        'ytick.minor.width':0.5,
        'lines.markeredgewidth':1,
        'axes.linewidth':1.0,
        'legend.fontsize': 7,
        'xtick.labelsize': 7,
        'ytick.labelsize': 7,
        'savefig.dpi':200,
        'path.simplify':True,
        #'font.family': 'serif',
        'font.serif':'Times',
        'text.latex.preamble': [r'\usepackage{amsmath}'],
        'text.usetex':True,
        #'axes.color_cycle': ['b', 'lime', 'r', 'purple', 'g', 'c', 'm', 'orange', 'darkblue', \
        #                        'darkcyan', 'y','orangered','chartreuse','brown','deeppink','lightgreen', 'k'],
        #'font.serif':cm,
        'figure.figsize': fig_size}
plt.rcParams.update(params)

In [5]:
RAs = []
DECs = []
Names = []
pmra = []
pmra_err = []
pmdec = []
pmdec_err = []
pepoch = []
posepoch = []
dmepoch = []
dm = []
coords = []

In [6]:
for file in files:
    d = {}
    with open(file) as f:
        for line in f:
            key = line.split()[0]
            value = line.split()[1]
            d[key] = value
            if key in ['RAJ', 'DECJ', 'PMRA', 'PMDEC', 'PX']:
                try:
                    err = line.split()[3]
                    d[key+'_ERR'] = err
                except:
                    print("this file is missing uncertainties \
on {0}: {1}".format(key, file))
                    d[key+'_ERR'] = 0.0
    
    try:
        ra = Angle(d['RAJ'], u.hourangle).to(u.deg)
        dec = Angle(d['DECJ'], u.degree)
        coord = SkyCoord(ra=ra, dec=dec, frame='icrs',
                         pm_ra_cosdec = float(d['PMRA'])*u.mas/u.yr,
                         pm_dec=float(d['PMDEC'])*u.mas/u.yr,
                         obstime=Time(float(d['POSEPOCH']), format='mjd'),
                         distance=1. * u.kpc) # HACK
        coords.append(coord)
        RAs.append(ra.value)
        DECs.append(dec.value)
        Names.append(d['PSRJ'])
        pmra.append(coord.pm_ra_cosdec.value)
        pmra_err.append(float(d['PMRA_ERR']))
        pmdec.append(coord.pm_dec.value)
        pmdec_err.append(float(d['PMDEC_ERR']))
        pepoch.append(d['PEPOCH'])
        posepoch.append(d['POSEPOCH'])
        dmepoch.append(d['DMEPOCH'])
        dm.append(d['DM'])
        
    except KeyError:
        lon = float(d['ELONG'])*u.deg
        lat = float(d['ELAT'])*u.deg
        coord = SkyCoord(l=lon, b=lat, frame='galactic',
                             pm_l_cosb = float(d['PMELONG'])*u.mas/u.yr, 
                             pm_b=float(d['PMELAT'])*u.mas/u.yr,
                             obstime=Time(float(d['POSEPOCH']), format='mjd'),
                             distance=1. * u.kpc) # HACKITY HACK HACK
        icrscoord = coord.transform_to(ICRS)
        coords.append(icrscoord)
        print("this object has galactic coords: {0}".format(d['PSRJ']))
        RAs.append(icrscoord.ra.to(u.deg).value)
        DECs.append(icrscoord.dec.to(u.deg).value)
        Names.append(d['PSRJ'])
        pmra.append(icrscoord.pm_ra_cosdec.value)
        pmra_err.append(0.0) # TEMPORARY HACK
        pmdec.append(icrscoord.pm_dec.value)
        pmdec_err.append(0.0) # TEMPORARY HACK
        pepoch.append(d['PEPOCH'])
        posepoch.append(d['POSEPOCH'])
        dmepoch.append(d['DMEPOCH'])
        dm.append(d['DM']) 

this file is missing uncertainties on RAJ: ../IPTA_DR1/J0437-4715/J0437-4715.par
this file is missing uncertainties on DECJ: ../IPTA_DR1/J0437-4715/J0437-4715.par
this file is missing uncertainties on PMRA: ../IPTA_DR1/J0437-4715/J0437-4715.par
this file is missing uncertainties on PMDEC: ../IPTA_DR1/J0437-4715/J0437-4715.par
this file is missing uncertainties on PX: ../IPTA_DR1/J0437-4715/J0437-4715.par
this object has galactic coords: J1022+1001
this file is missing uncertainties on RAJ: ../IPTA_DR1/J1643-1224/J1643-1224.par
this file is missing uncertainties on DECJ: ../IPTA_DR1/J1643-1224/J1643-1224.par
this file is missing uncertainties on PMRA: ../IPTA_DR1/J1643-1224/J1643-1224.par
this file is missing uncertainties on PMDEC: ../IPTA_DR1/J1643-1224/J1643-1224.par
this file is missing uncertainties on PX: ../IPTA_DR1/J1643-1224/J1643-1224.par


In [ ]:
#Names

### J1910+1256

In [ ]:
ind = np.where(np.array(Names) == 'J1910+1256')[0][0]

In [ ]:
ipta_pmra_err = pmra_err[ind]
ipta_pmdec_err = pmdec_err[ind]
ipta_posepoch = posepoch[ind]

In [ ]:
ipta_coord = coords[ind]
ipta_coord

In [ ]:
gaia_time = Time(57023., format='mjd')
gaia_match_coord =  SkyCoord(ra=287.5407215 * u.degree, 
                             dec=12.94037301 * u.degree, frame='icrs',
                         pm_ra_cosdec = -0.672057539 *u.mas/u.yr,
                         pm_dec= -5.449112763 *u.mas/u.yr,
                         obstime=Time(2015.5, format='decimalyear'),
                         distance=1. * u.kpc) # HACK
gaia_ra_err = 0.576369178
gaia_dec_err = 0.623526292
gaia_match_coord

In [ ]:
N = 6000
seps = []
ipta_ras = []
ipta_decs = []
gaia_ras = []
gaia_decs = []
for n in tqdm(range(N)):
    c1 = SkyCoord(ra=ipta_coord.ra,
                 dec=ipta_coord.dec, frame='icrs',
                 pm_ra_cosdec=np.random.normal(ipta_coord.pm_ra_cosdec.value, 
                                               ipta_pmra_err) * u.mas/u.yr,
                 pm_dec=np.random.normal(ipta_coord.pm_dec.value, 
                                               ipta_pmdec_err) * u.mas/u.yr,
                 obstime=ipta_coord.obstime,
                 distance=ipta_coord.distance)
    c2 = SkyCoord(ra=np.random.normal(gaia_match_coord.ra.arcsec, gaia_ra_err) * u.arcsec,
                 dec=np.random.normal(gaia_match_coord.dec.arcsec, gaia_dec_err) * u.arcsec, 
                 frame='icrs',
                 pm_ra_cosdec=gaia_match_coord.pm_ra_cosdec, 
                 pm_dec=gaia_match_coord.pm_dec, 
                 obstime=gaia_match_coord.obstime,
                 distance=gaia_match_coord.distance)
    ipta_ras.append(c1.apply_space_motion(new_obstime=gaia_time).ra.degree)
    ipta_decs.append(c1.apply_space_motion(new_obstime=gaia_time).dec.degree)
    gaia_ras.append(c2.ra.degree)
    gaia_decs.append(c2.dec.degree)
    sep = c1.apply_space_motion(new_obstime=gaia_time).separation(c2).arcsec
    seps.append(sep)

In [ ]:
gaia_ras = np.array(gaia_ras)-287.54
ipta_ras = np.array(ipta_ras)-287.54


In [ ]:
bin_list = auto_bin(np.std(gaia_ras)/5,np.min(gaia_ras)-np.std(gaia_ras),np.max(gaia_ras)+np.std(gaia_ras))
weights = hist_weight(gaia_ras)

In [ ]:
np.std(gaia_ras)

In [ ]:
(mu, sigma) = norm.fit(gaia_ras)
diff = (mu-np.median(ipta_ras))/sigma

In [ ]:
(mu, sigma)

In [ ]:
diff

In [ ]:
# add a 'best fit' line
y = norm.fit(gaia_ras)

y = scipy.stats.norm.pdf(bin_list, mu, sigma)

l = plt.plot(bin_list, y, 'r--', linewidth=2)

In [ ]:
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

In [ ]:
normed = []
for xx in range(len(bin_list)):
    normed.append(gaussian(bin_list[xx], mu, sigma))

In [ ]:
datos = gaia_ras
(mu, sigma) = norm.fit(datos)

# the histogram of the data
n, bins, patches = plt.hist(datos, 20, normed=.12, facecolor='green', alpha=0.75)

# add a 'best fit' line
y = scipy.stats.norm.pdf(bins, mu, sigma)

l = plt.plot(bins, y, 'r--', linewidth=2)
plt.axvline(np.median(ipta_ras), color='red', label = "IPTA")



In [ ]:
ipta_ras = np.array(ipta_ras)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.hist(np.array(gaia_ras), bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(np.median(ipta_ras), color='red', label = "IPTA")
#y = mlab.normpdf(bin_list, mu, sigma)
#plt.plot(bin_list, normed, 'r--', linewidth=2, label=r"Gaussian Fit")
plt.xlabel("RA - 287.54 (deg)")
plt.yticks([])
#plt.ylabel("Probability")
plt.legend(frameon=False)
plt.tight_layout()
plt.savefig("J1910_RA_IPTA-GAIA-final-24Nov18.pdf")

In [ ]:
2.8754e2

In [ ]:
sigmaRA

In [ ]:
bin_list = auto_bin(np.std(gaia_decs)/4,np.min(gaia_decs)-np.std(gaia_decs),np.max(gaia_decs)+np.std(gaia_decs))
weights = hist_weight(gaia_decs)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.hist(gaia_decs, bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(np.median(ipta_decs), color='red', label = "IPTA") #0.2 sigma
plt.xlabel("DEC (deg)")
plt.yticks([])
#plt.ylabel("Probability")
plt.legend(frameon=False)
plt.tight_layout()
#plt.savefig("J1910_DEC_IPTA-GAIA-final-25Nov.pdf")

In [ ]:
(mu, sigma) = norm.fit(gaia_decs)
diff = (mu-np.median(ipta_decs))/sigma

In [ ]:
diff

##  J1012+5307

In [ ]:
ind = np.where(np.array(Names) == 'J1012+5307')[0][0]

In [ ]:
ipta_pmra_err = pmra_err[ind]
ipta_pmdec_err = pmdec_err[ind]
ipta_posepoch = posepoch[ind]

In [ ]:
ipta_coord = coords[ind]
ipta_coord

In [ ]:
gaia_time = Time(57023., format='mjd')
gaia_match_coord =  SkyCoord(ra=153.1393303 * u.degree, 
                             dec=53.11726303 * u.degree, frame='icrs',
                         pm_ra_cosdec = 2.976959985 *u.mas/u.yr,
                         pm_dec= -26.94384758 *u.mas/u.yr,
                         obstime= gaia_time,
                         distance=1. * u.kpc) # HACK
gaia_ra_err = 0.241174616
gaia_dec_err = 0.302546482
gaia_match_coord

In [ ]:
N = 5000
seps = []
ipta_ras = []
ipta_decs = []
gaia_ras = []
gaia_decs = []
for n in tqdm(range(N)):
    c1 = SkyCoord(ra=ipta_coord.ra,
                 dec=ipta_coord.dec, frame='icrs',
                 pm_ra_cosdec=np.random.normal(ipta_coord.pm_ra_cosdec.value, 
                                               ipta_pmra_err) * u.mas/u.yr,
                 pm_dec=np.random.normal(ipta_coord.pm_dec.value, 
                                               ipta_pmdec_err) * u.mas/u.yr,
                 obstime=ipta_coord.obstime,
                 distance=ipta_coord.distance)
    c2 = SkyCoord(ra=np.random.normal(gaia_match_coord.ra.arcsec, gaia_ra_err) * u.arcsec,
                 dec=np.random.normal(gaia_match_coord.dec.arcsec, gaia_dec_err) * u.arcsec, 
                 frame='icrs',
                 pm_ra_cosdec=gaia_match_coord.pm_ra_cosdec, 
                 pm_dec=gaia_match_coord.pm_dec, 
                 obstime=gaia_match_coord.obstime,
                 distance=gaia_match_coord.distance)
    ipta_ras.append(c1.apply_space_motion(new_obstime=gaia_time).ra.degree)
    ipta_decs.append(c1.apply_space_motion(new_obstime=gaia_time).dec.degree)
    gaia_ras.append(c2.ra.degree)
    gaia_decs.append(c2.dec.degree)
    sep = c1.apply_space_motion(new_obstime=gaia_time).separation(c2).arcsec
    seps.append(sep)

In [ ]:
gaia_ras = np.array(gaia_ras) -1.53139e2 #for better plotting

In [ ]:
bin_list = auto_bin(np.std(gaia_ras)/4,np.min(gaia_ras)-np.std(gaia_ras)/2,np.max(gaia_ras)+np.std(gaia_ras)/2)
weights = hist_weight(gaia_ras)

In [ ]:
(mu, sigma) = norm.fit(gaia_ras)
diff = (mu-np.median(ipta_ras))/sigma

In [ ]:
mu

In [ ]:
diff

In [ ]:
ipta_ras = np.array(ipta_ras) -153.139

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.hist(np.array(gaia_ras), bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(np.median(ipta_ras), color='red', label = "PTA")
plt.xlabel("RA + 153.139 (deg)")
plt.yticks([])
plt.legend(frameon=False)
plt.tight_layout()
#plt.savefig("J1012_RA_IPTA-GAIA-25Nov.pdf")

In [ ]:
gaia_decs=np.array(gaia_decs) -53.117

In [ ]:
gaia_decs

In [ ]:
bin_list = auto_bin(np.std(gaia_decs)/4,np.min(gaia_decs)-np.std(gaia_decs)/2,np.max(gaia_decs)+np.std(gaia_decs)/2)
weights = hist_weight(gaia_decs)

In [ ]:
(mu, sigma) = norm.fit(gaia_decs)
diff = (mu-np.median(ipta_decs))/sigma

In [ ]:
ipta_decs = np.array(ipta_decs) - 53.117

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.hist(np.array(gaia_decs), bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(np.median(ipta_decs), color='red', label = "IPTA")
plt.xlabel("DEC + 53.117 (deg)")
plt.yticks([])
plt.legend(frameon=False)
plt.tight_layout()
#plt.savefig("J1012_DEC_IPTA-GAIA-25Nov.pdf")

## Best pulasr, J0437-4715


In [ ]:
ind = np.where(np.array(Names) == 'J0437-4715')[0][0]

In [ ]:
ipta_pmra_err = pmra_err[ind]
ipta_pmdec_err = pmdec_err[ind]
ipta_posepoch = posepoch[ind]

In [ ]:
ipta_pmra_err = 0.052 #Deller et al. 2008, Table 1
ipta_pmdec_err = 0.086 #Deller et al. 2008
ipta_posepoch = posepoch[ind]

In [ ]:
ipta_posepoch

In [ ]:
arcSec2Deg = 0.000277778

In [ ]:
ipta_coord = coords[ind]
ipta_coord

In [ ]:
gaia_time = Time(57023., format='mjd') # CHECK THAT THIS IS REALLY 2015.5 IN MJD
gaia_match_coord =  SkyCoord(ra=69.31660276 * u.degree, 
                             dec=-47.25267778 * u.degree, frame='icrs',
                         pm_ra_cosdec = 122.8637296 *u.mas/u.yr,
                         pm_dec= -71.16597382 *u.mas/u.yr,
                         obstime=Time(2015.5, format='decimalyear'),
                         distance=1. * u.kpc) # HACK
gaia_ra_err = 0.582326756
gaia_dec_err = 0.738504992
gaia_match_coord

In [ ]:
N = 6000
seps = []
ipta_ras = []
ipta_decs = []
gaia_ras = []
gaia_decs = []
for n in tqdm(range(N)):
    c1 = SkyCoord(ra=ipta_coord.ra,
                 dec=ipta_coord.dec, frame='icrs',
                 pm_ra_cosdec=np.random.normal(ipta_coord.pm_ra_cosdec.value, 
                                               ipta_pmra_err) * u.mas/u.yr,
                 pm_dec=np.random.normal(ipta_coord.pm_dec.value, 
                                               ipta_pmdec_err) * u.mas/u.yr,
                 obstime=ipta_coord.obstime,
                 distance=ipta_coord.distance)
    c2 = SkyCoord(ra=np.random.normal(gaia_match_coord.ra.arcsec, gaia_ra_err) * u.arcsec,
                 dec=np.random.normal(gaia_match_coord.dec.arcsec, gaia_dec_err) * u.arcsec, 
                 frame='icrs',
                 pm_ra_cosdec=gaia_match_coord.pm_ra_cosdec, 
                 pm_dec=gaia_match_coord.pm_dec, 
                 obstime=gaia_match_coord.obstime,
                 distance=gaia_match_coord.distance)
    ipta_ras.append(c1.apply_space_motion(new_obstime=gaia_time).ra.degree)
    #print(ipta_ras)
    ipta_decs.append(c1.apply_space_motion(new_obstime=gaia_time).dec.degree)
    gaia_ras.append(c2.ra.degree)
    gaia_decs.append(c2.dec.degree)
    sep = c1.apply_space_motion(new_obstime=gaia_time).separation(c2).arcsec
    seps.append(sep)

In [ ]:
ipta_ras = np.array(ipta_ras) - 69.31
gaia_ras = np.array(gaia_ras) - 69.31

In [ ]:
bin_list = auto_bin(np.std(gaia_ras)/5,np.min(gaia_ras)-np.std(gaia_ras)/2,np.max(gaia_ras)+np.std(gaia_ras)/2)
weights = hist_weight(gaia_ras)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.hist(gaia_ras, bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(np.median(ipta_ras), color='red', label = "IPTA") #0.13 sigma
plt.xlabel("RA - 69.31(deg)")
plt.yticks([])
plt.legend(frameon=False)
plt.tight_layout()
#plt.savefig("J0437_RA_IPTA-GAIA-25Nov.pdf")

In [ ]:
(mu, sigma) = norm.fit(gaia_ras)
diff = (mu-np.median(ipta_ras))/sigma

In [ ]:
diff

In [ ]:
bin_list = auto_bin(np.std(gaia_decs)/4,np.min(gaia_decs)-np.std(gaia_decs)/2,np.max(gaia_decs)+np.std(gaia_decs)/2)
weights = hist_weight(gaia_decs)

In [ ]:
ipta_decs = np.array(ipta_decs) - 47.25
gaia_decs = np.array(gaia_decs) - 47.25

In [ ]:
ipta_decs

In [ ]:
# DEC plot for J0437
plt.hist(gaia_decs, bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(np.median(ipta_decs), color='red', label = "IPTA")
plt.xlabel("DEC - 47.25(deg)")
plt.yticks([])
plt.legend(frameon=False)
plt.tight_layout()
#plt.savefig("J0437_DEC_IPTA-GAIA-25Nov.pdf")

In [ ]:
(mu, sigma) = norm.fit(gaia_decs)
diff = (mu-np.median(ipta_decs))/sigma

In [ ]:
diff

In [ ]:
ipta_decs + 47.25

## Pulsar J1024-0719

In [ ]:
ind = np.where(np.array(Names) == 'J1024-0719')[0][0]

ipta_pmra_err = pmra_err[ind]
ipta_pmdec_err = pmdec_err[ind]
ipta_posepoch = posepoch[ind]

ipta_pmra_err = pmra_err[ind]
ipta_pmdec_err = pmdec_err[ind]
ipta_posepoch = posepoch[ind]


ipta_coord = coords[ind]
ipta_coord

In [ ]:
gaia_time = Time(57023., format='mjd')
gaia_match_coord =  SkyCoord(ra=156.1610927 * u.degree, 
                             dec=-7.322165289 * u.degree, frame='icrs',
                         pm_ra_cosdec = -35.51929892*u.mas/u.yr,
                         pm_dec= -47.93231066 *u.mas/u.yr,
                         obstime=Time(2015.5, format='decimalyear'),
                         distance=1. * u.kpc) # HACK
gaia_ra_err = 0.42985265
gaia_dec_err = 0.392656637
gaia_match_coord

In [ ]:
N = 6000
seps = []
ipta_ras = []
ipta_decs = []
gaia_ras = []
gaia_decs = []
for n in tqdm(range(N)):
    c1 = SkyCoord(ra=ipta_coord.ra,
                 dec=ipta_coord.dec, frame='icrs',
                 pm_ra_cosdec=np.random.normal(ipta_coord.pm_ra_cosdec.value, 
                                               ipta_pmra_err) * u.mas/u.yr,
                 pm_dec=np.random.normal(ipta_coord.pm_dec.value, 
                                               ipta_pmdec_err) * u.mas/u.yr,
                 obstime=ipta_coord.obstime,
                 distance=ipta_coord.distance)
    c2 = SkyCoord(ra=np.random.normal(gaia_match_coord.ra.arcsec, gaia_ra_err) * u.arcsec,
                 dec=np.random.normal(gaia_match_coord.dec.arcsec, gaia_dec_err) * u.arcsec, 
                 frame='icrs',
                 pm_ra_cosdec=gaia_match_coord.pm_ra_cosdec, 
                 pm_dec=gaia_match_coord.pm_dec, 
                 obstime=gaia_match_coord.obstime,
                 distance=gaia_match_coord.distance)
    ipta_ras.append(c1.apply_space_motion(new_obstime=gaia_time).ra.degree)
    ipta_decs.append(c1.apply_space_motion(new_obstime=gaia_time).dec.degree)
    gaia_ras.append(c2.ra.degree)
    gaia_decs.append(c2.dec.degree)
    sep = c1.apply_space_motion(new_obstime=gaia_time).separation(c2).arcsec
    seps.append(sep)

In [ ]:
gaia_ras = np.array(gaia_ras) + 1.5616e2

In [ ]:
ipta_ras = np.array(ipta_ras) + 1.5616e2

In [ ]:
bin_list = auto_bin(np.std(gaia_ras)/4,np.min(gaia_ras)-np.std(gaia_ras)/2,np.max(gaia_ras)+np.std(gaia_ras)/2)
weights = hist_weight(gaia_ras)

In [ ]:
(mu, sigma) = norm.fit(gaia_ras)
diff = (mu-np.median(ipta_ras))/sigma

In [ ]:
diff

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.hist(gaia_ras, bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(np.median(ipta_ras), color='red', label = "IPTA") #0.06 sigma, great fit!
plt.xlabel("RA + 156.16 (deg)")
plt.yticks([])
plt.legend(frameon=False)
plt.tight_layout()
#plt.savefig("J1024_RA_IPTA-GAIA.pdf")

In [ ]:
bin_list = auto_bin(np.std(gaia_decs)/4,np.min(gaia_decs)-np.std(gaia_decs)/2,np.max(gaia_decs)+np.std(gaia_decs)/2)
weights = hist_weight(gaia_decs)


In [ ]:
(mu, sigma) = norm.fit(gaia_decs)
diff = (mu-np.median(ipta_decs))/sigma



In [ ]:
diff

In [ ]:
# DEC plot for J1024
plt.hist(gaia_decs, bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(np.median(ipta_decs), color='red', label = "IPTA") #0.2 sigma away
plt.xlabel("DEC (deg)")
plt.yticks([])
plt.legend(frameon=False)
plt.tight_layout()
#plt.savefig("J1024_DEC_IPTA-GAIA-25Nov.pdf")

## J1732-5049

In [ ]:
ind = np.where(np.array(Names) == 'J1732-5049')[0][0]

ipta_pmra_err = pmra_err[ind]
ipta_pmdec_err = pmdec_err[ind]
ipta_posepoch = posepoch[ind]

ipta_pmra_err = pmra_err[ind]
ipta_pmdec_err = pmdec_err[ind]
ipta_posepoch = posepoch[ind]


ipta_coord = coords[ind]
ipta_coord

In [ ]:
gaia_time = Time(57023., format='mjd')
gaia_match_coord =  SkyCoord(ra=263.200006 * u.degree, 
                             dec=-50.8166163 * u.degree, frame='icrs',
                         pm_ra_cosdec = -0.031849016*u.mas/u.yr,
                         pm_dec= -10.483062*u.mas/u.yr,
                         obstime=Time(2015.5, format='decimalyear'),
                         distance=1. * u.kpc) # HACK
gaia_ra_err = 2.168898313
gaia_dec_err = 2.010098257
gaia_match_coord

In [ ]:
gaia_ipta_sep = ipta_coord.apply_space_motion(new_obstime=gaia_time).separation(gaia_match_coord).arcsec
gaia_ipta_sep

In [ ]:
N = 6000
seps = []
ipta_ras = []
ipta_decs = []
gaia_ras = []
gaia_decs = []
for n in tqdm(range(N)):
    c1 = SkyCoord(ra=ipta_coord.ra,
                 dec=ipta_coord.dec, frame='icrs',
                 pm_ra_cosdec=np.random.normal(ipta_coord.pm_ra_cosdec.value, 
                                               ipta_pmra_err) * u.mas/u.yr,
                 pm_dec=np.random.normal(ipta_coord.pm_dec.value, 
                                               ipta_pmdec_err) * u.mas/u.yr,
                 obstime=ipta_coord.obstime,
                 distance=ipta_coord.distance)
    c2 = SkyCoord(ra=np.random.normal(gaia_match_coord.ra.arcsec, gaia_ra_err) * u.arcsec,
                 dec=np.random.normal(gaia_match_coord.dec.arcsec, gaia_dec_err) * u.arcsec, 
                 frame='icrs',
                 pm_ra_cosdec=gaia_match_coord.pm_ra_cosdec, 
                 pm_dec=gaia_match_coord.pm_dec, 
                 obstime=gaia_match_coord.obstime,
                 distance=gaia_match_coord.distance)
    ipta_ras.append(c1.apply_space_motion(new_obstime=gaia_time).ra.degree)
    ipta_decs.append(c1.apply_space_motion(new_obstime=gaia_time).dec.degree)
    gaia_ras.append(c2.ra.degree)
    gaia_decs.append(c2.dec.degree)
    sep = c1.apply_space_motion(new_obstime=gaia_time).separation(c2).arcsec
    seps.append(sep)

In [ ]:
bin_list = auto_bin(.5,0.0,8.0)
weights = hist_weight(seps)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.hist(seps, bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(gaia_ipta_sep, color='red', label = "IPTA")
plt.xlabel(r"J1732-5049, Angular Separation $\theta$ (arcsecs)")
plt.ylabel("Probability")
plt.legend(frameon=False)
plt.tight_layout()
#plt.savefig("J1732_Theta_IPTA-GAIA.pdf")

In [ ]:
ipta_ras = np.array(ipta_ras)-263.2

In [ ]:
gaia_ras = np.array(gaia_ras)-263.2

In [ ]:
bin_list = auto_bin(np.std(gaia_ras)/4,np.min(gaia_ras)-np.std(gaia_ras)/2,np.max(gaia_ras)+np.std(gaia_ras)/2)
weights = hist_weight(gaia_ras)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.hist(gaia_ras, bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(np.median(ipta_ras), color='red', label = "IPTA") #1.57 sigma
plt.xlabel("RA - 263.2 (deg)")
plt.yticks([])
plt.legend(frameon=False)
plt.tight_layout()
plt.savefig("J1732_RA_IPTA-GAIA-25Nov.pdf")

In [ ]:
(mu, sigma) = norm.fit(gaia_ras)
diff = (mu-np.median(ipta_ras))/sigma

In [ ]:
mu

In [ ]:
sigma

In [ ]:
diff

In [ ]:
bin_list = auto_bin(np.std(gaia_decs)/4,np.min(gaia_decs)-np.std(gaia_decs)/2,np.max(gaia_decs)+np.std(gaia_decs)/2)
weights = hist_weight(gaia_decs)

In [ ]:
gaia_decs = np.array(gaia_decs) - 50.81

In [ ]:
ipta_decs = np.array(ipta_decs) - 50.81

In [ ]:
gaia_decs

In [ ]:
# DEC plot for J1024
plt.hist(gaia_decs, bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(np.median(ipta_decs), color='red', label = "IPTA") #0.26 sigma away
#ax.set_xlim([np.min(ipta_decs),np.max(ipta_decs)])

plt.xlabel("DEC + 50.81 (deg)")
plt.yticks([])
plt.legend(frameon=False)
plt.tight_layout()
#plt.savefig("J1732_DEC_IPTA-GAIA-25Nov.pdf")

In [ ]:
(mu, sigma) = norm.fit(gaia_decs)
diff = (mu-np.median(ipta_decs))/sigma

In [ ]:
diff

## Pulsar J1955+2908

In [ ]:
ind = np.where(np.array(Names) == 'J1955+2908')[0][0]

ipta_pmra_err = pmra_err[ind]
ipta_pmdec_err = pmdec_err[ind]
ipta_posepoch = posepoch[ind]

ipta_pmra_err = pmra_err[ind]
ipta_pmdec_err = pmdec_err[ind]
ipta_posepoch = posepoch[ind]


ipta_coord = coords[ind]
ipta_coord

In [ ]:
gaia_time = Time(57023., format='mjd')
gaia_match_coord =  SkyCoord(ra=298.8661015 * u.degree, 
                             dec=29.14538275 * u.degree, frame='icrs',
                         pm_ra_cosdec = -3.092116051*u.mas/u.yr,
                         pm_dec= -7.962991888*u.mas/u.yr,
                         obstime=Time(2015.5, format='decimalyear'),
                         distance=1. * u.kpc) # HACK
gaia_ra_err = 0.125519342
gaia_dec_err = 0.165511548
gaia_match_coord

In [ ]:
gaia_ipta_sep = ipta_coord.apply_space_motion(new_obstime=gaia_time).separation(gaia_match_coord).arcsec
gaia_ipta_sep

In [ ]:
N = 6000
seps = []
ipta_ras = []
ipta_decs = []
gaia_ras = []
gaia_decs = []
for n in tqdm(range(N)):
    c1 = SkyCoord(ra=ipta_coord.ra,
                 dec=ipta_coord.dec, frame='icrs',
                 pm_ra_cosdec=np.random.normal(ipta_coord.pm_ra_cosdec.value, 
                                               ipta_pmra_err) * u.mas/u.yr,
                 pm_dec=np.random.normal(ipta_coord.pm_dec.value, 
                                               ipta_pmdec_err) * u.mas/u.yr,
                 obstime=ipta_coord.obstime,
                 distance=ipta_coord.distance)
    c2 = SkyCoord(ra=np.random.normal(gaia_match_coord.ra.arcsec, gaia_ra_err) * u.arcsec,
                 dec=np.random.normal(gaia_match_coord.dec.arcsec, gaia_dec_err) * u.arcsec, 
                 frame='icrs',
                 pm_ra_cosdec=gaia_match_coord.pm_ra_cosdec, 
                 pm_dec=gaia_match_coord.pm_dec, 
                 obstime=gaia_match_coord.obstime,
                 distance=gaia_match_coord.distance)
    ipta_ras.append(c1.apply_space_motion(new_obstime=gaia_time).ra.degree)
    ipta_decs.append(c1.apply_space_motion(new_obstime=gaia_time).dec.degree)
    gaia_ras.append(c2.ra.degree)
    gaia_decs.append(c2.dec.degree)
    sep = c1.apply_space_motion(new_obstime=gaia_time).separation(c2).arcsec
    seps.append(sep)

In [ ]:
bin_list = auto_bin(.05,0.0,0.8)
weights = hist_weight(seps)

fig = plt.figure()
ax = fig.add_subplot(111)
plt.hist(seps, bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(gaia_ipta_sep, color='red', label = "IPTA")
plt.xlabel(r"J1955+2908, Angular Separation $\theta$ (arcsecs)")
plt.ylabel("Probability")
plt.legend(frameon=False)
plt.tight_layout()
#plt.savefig("J1955_Theta_IPTA-GAIA.pdf")

In [ ]:
ipta_ras = np.array(ipta_ras) - 298.85

In [ ]:
gaia_ras = np.array(gaia_ras) - 298.85

In [ ]:
bin_list = auto_bin(np.std(gaia_ras)/4,np.min(gaia_ras)-np.std(gaia_ras)/2,np.max(gaia_ras)+np.std(gaia_ras)/2)
weights = hist_weight(gaia_ras)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.hist(gaia_ras, bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(np.median(ipta_ras), color='red', label = "IPTA") #1.3 sigma
plt.xlabel("RA - 298.85 (deg)")
plt.yticks([])
plt.legend(frameon=False)
plt.tight_layout()
plt.savefig("J1955_RA_IPTA-GAIA-25Nov.pdf")

In [ ]:
(mu, sigma) = norm.fit(gaia_ras)
diff = (mu-np.median(ipta_ras))/sigma

In [ ]:
diff

In [ ]:
gaia_decs = np.array(gaia_decs) - 29.145

In [ ]:
ipta_decs = np.array(ipta_decs) - 29.145

In [ ]:
max(gaia_decs)

In [ ]:
bin_list = auto_bin(np.std(gaia_decs)/4,np.min(gaia_decs)-np.std(gaia_decs)/2,np.max(gaia_decs)+np.std(gaia_decs)/2)
weights = hist_weight(gaia_decs)

In [ ]:
# DEC plot for J1732
plt.hist(gaia_decs, bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(np.median(ipta_decs), color='red', label = "IPTA") #0.4 sigma
ax.set_xlim([np.min(ipta_decs),np.max(ipta_decs)])
plt.xlabel("DEC - 29.145 (deg)")
plt.legend(frameon=False)
plt.yticks([])
plt.tight_layout()
#plt.savefig("J1955_DEC_IPTA-GAIA-25Nov.pdf")

In [ ]:
(mu, sigma) = norm.fit(gaia_decs)
diff = (mu-np.median(ipta_decs))/sigma


# J1843-1113

In [7]:
## Pulsar J1843-1113

ind = np.where(np.array(Names) == 'J1843-1113')[0][0]

ipta_pmra_err = pmra_err[ind]
ipta_pmdec_err = pmdec_err[ind]
ipta_posepoch = posepoch[ind]

ipta_pmra_err = pmra_err[ind]
ipta_pmdec_err = pmdec_err[ind]
ipta_posepoch = posepoch[ind]


ipta_coord = coords[ind]
ipta_coord

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    (280.92192463, -11.22529681, 1.)
 (pm_ra_cosdec, pm_dec) in mas / yr
    (-1.85612422, -3.7551222)>

In [8]:
gaia_time = Time(57023., format='mjd')
gaia_match_coord =  SkyCoord(ra=280.9223272 * u.degree, 
                             dec=-11.22530736 * u.degree, frame='icrs',
                         pm_ra_cosdec = -3.241264413*u.mas/u.yr,
                         pm_dec= -7.274383834*u.mas/u.yr,
                         obstime=Time(2015.5, format='decimalyear'),
                         distance=1. * u.kpc) # HACK
gaia_ra_err = 0.666668976
gaia_dec_err = 0.644355643
gaia_match_coord

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    (280.9223272, -11.22530736, 1.)
 (pm_ra_cosdec, pm_dec) in mas / yr
    (-3.24126441, -7.27438383)>

In [9]:
gaia_ipta_sep = ipta_coord.apply_space_motion(new_obstime=gaia_time).separation(gaia_match_coord).arcsec
gaia_ipta_sep

1.4319161444709916

In [10]:
N = 6000
seps = []
ipta_ras = []
ipta_decs = []
gaia_ras = []
gaia_decs = []
for n in tqdm(range(N)):
    c1 = SkyCoord(ra=ipta_coord.ra,
                 dec=ipta_coord.dec, frame='icrs',
                 pm_ra_cosdec=np.random.normal(ipta_coord.pm_ra_cosdec.value, 
                                               ipta_pmra_err) * u.mas/u.yr,
                 pm_dec=np.random.normal(ipta_coord.pm_dec.value, 
                                               ipta_pmdec_err) * u.mas/u.yr,
                 obstime=ipta_coord.obstime,
                 distance=ipta_coord.distance)
    c2 = SkyCoord(ra=np.random.normal(gaia_match_coord.ra.arcsec, gaia_ra_err) * u.arcsec,
                 dec=np.random.normal(gaia_match_coord.dec.arcsec, gaia_dec_err) * u.arcsec, 
                 frame='icrs',
                 pm_ra_cosdec=gaia_match_coord.pm_ra_cosdec, 
                 pm_dec=gaia_match_coord.pm_dec, 
                 obstime=gaia_match_coord.obstime,
                 distance=gaia_match_coord.distance)
    ipta_ras.append(c1.apply_space_motion(new_obstime=gaia_time).ra.degree)
    ipta_decs.append(c1.apply_space_motion(new_obstime=gaia_time).dec.degree)
    gaia_ras.append(c2.ra.degree)
    gaia_decs.append(c2.dec.degree)
    sep = c1.apply_space_motion(new_obstime=gaia_time).separation(c2).arcsec
    seps.append(sep)

100%|██████████| 6000/6000 [13:32<00:00,  7.38it/s]


In [ ]:
bin_list = auto_bin(.05,0.0,0.8)
weights = hist_weight(seps)

fig = plt.figure()
ax = fig.add_subplot(111)
plt.hist(seps, bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(gaia_ipta_sep, color='red', label = "IPTA")
plt.xlabel(r"J1955+2908, Angular Separation $\theta$ (arcsecs)")
plt.ylabel("Probability")
plt.legend(frameon=False)
plt.tight_layout()
#plt.savefig("J1955_Theta_IPTA-GAIA.pdf")

In [ ]:
ipta_ras = np.array(ipta_ras) -280.92 # minus some number which makes plotting easier


In [ ]:
gaia_ras = np.array(gaia_ras) -280.92 # minus some number which makes plotting easier

In [ ]:
bin_list = auto_bin(np.std(gaia_ras)/4,np.min(gaia_ras)-np.std(gaia_ras)/2,np.max(gaia_ras)+np.std(gaia_ras)/2)
weights = hist_weight(gaia_ras)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.hist(gaia_ras, bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(np.median(ipta_ras), color='red', label = "IPTA") #1.3 sigma
plt.xlabel("RA + 280.92 (deg)")
plt.yticks([])
plt.legend(frameon=False)
plt.tight_layout()
#plt.savefig("J1843_RA_IPTA-GAIA-29Nov.pdf")

In [ ]:
(mu, sigma) = norm.fit(gaia_ras)
diff = (mu-np.median(ipta_ras))/sigma

In [ ]:
diff

In [ ]:
gaia_decs = np.array(gaia_decs) +11.22#
ipta_decs = np.array(ipta_decs)+11.22

In [ ]:
bin_list = auto_bin(np.std(gaia_decs)/4,np.min(gaia_decs)-np.std(gaia_decs)/2,np.max(gaia_decs)+np.std(gaia_decs)/2)
weights = hist_weight(gaia_decs)

In [ ]:
# DEC plot for J1843
plt.hist(gaia_decs, bin_list, weights=weights, alpha = 0.8, align='left', color = 'C0', label = "Gaia")
plt.axvline(np.median(ipta_decs), color='red', label = "IPTA") #0.4 sigma
ax.set_xlim([np.min(ipta_decs),np.max(ipta_decs)])
plt.xlabel("DEC - 11.22 (deg)")
plt.legend(frameon=False)
plt.yticks([])
plt.tight_layout()
#plt.savefig("J1843_DEC_IPTA-GAIA-25Nov.pdf")